<a href="https://colab.research.google.com/github/ssroka/ML_with_IBM_GeoDN/blob/main/NDVI_for_map_county.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IBM EIS PAIRS GeoDN
https://environmentalintelligencesuite.ibm.com

# install packages for basemap

In [ ]:
!pip install folium==0.2.1
!pip install ibmpairs
!pip install geopandas

!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install "basemap == 1.3.0b1" "basemap-data == 1.3.0b1"

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79813 sha256=4bf4974b4e7ee2cf87677ee6ffd330b4591840d137f10e3141e1db373182ee3c
  Stored in directory: /root/.cache/pip/wheels/24/be/74/8b4e42555090b2aad019a42e9d354060820e2fffb8c48e44a7
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.12.1.post1
    Uninstalling folium-0.12.1.post1:
      Successfully uninstalled folium-0.12.1.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.5 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 9.3 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libgeos-3.5.0
E: Couldn't find any package by glo

Mounted at /content/drive


# import packages

In [ ]:
# PAIRS packages
import ibmpairs
from   ibmpairs import paw, authentication
import ibmpairs.authentication as authentication
import ibmpairs.client as client
import ibmpairs.query as query
import ibmpairs.catalog as catalog
import requests

# map-related packages
import geopandas
import PIL.Image # to convert PAIRS geotiff to numpy arrays
from mpl_toolkits.basemap import Basemap
import json

# the usual suspects
import pandas as pd
import numpy as np
import shutil
import os
import time
from datetime import datetime, timedelta
import pickle
from re import L


# getting fancy
import threading

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits

# saving
from csv import writer


## GeoDN Credentials

In [ ]:
key_file = '/content/drive/My Drive/Colab Notebooks/PAIRS_KEY.txt'
with open(key_file, "r") as f:
  EIS_APIKEY = f.read()

name_file = '/content/drive/My Drive/Colab Notebooks/PAIRS_USERNAME.txt'
with open(name_file, "r") as f:
  EIS_USERNAME = f.read()

PAIRS_SERVER   = 'https://pairs.res.ibm.com'
PAIRS_CREDENTIALS = authentication.OAuth2(api_key = EIS_APIKEY)

# Create an authentication object with credentials.
credentials  = authentication.OAuth2(username = EIS_USERNAME,
                                     api_key  = EIS_APIKEY)

# Add the credentials object to a client object.
eis_client = client.Client(authentication = credentials)

## Read in county number and coordinates from MATLAB-produced text file

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/CCUS_map/countyns.py') as f:
  Lines = f.readlines()

print(Lines[0].split("'")[3])
print(Lines[0].split("'")[7])

N_cnty = len(Lines)


00350547
(-85.14 31.84,-85.11 31.89,-85.07 31.97,-85.06 31.99,-85.04 31.98,-84.97 31.95,-84.91 31.92,-84.91 31.78,-84.96 31.78,-85.07 31.77,-85.13 31.78,-85.14 31.84)


In [ ]:
# find the centroid of each county
cent = []
for i in range(N_cnty):
  # remove ()'s 
  coords = Lines[i].split("'")[7][1:-1].split(",")
  Lon = []
  Lat = []
  for j in range(len(coords)):
    Lon.append(float(coords[j].split(" ")[0]))
    Lat.append(float(coords[j].split(" ")[1]))
  cent.append([np.mean(np.array(Lon)),np.mean(np.array(Lat))])

In [ ]:
cns = []
crds = []
for i in range(N_cnty):
  cns.append(Lines[i].split("'")[3])  
  # crds.append(Lines[i].split("'")[7])

In [ ]:
df = pd.DataFrame()

df['CountyNS'] = cns
df['Coords']   = cent
df['NDVI']     = np.zeros((N_cnty))

In [ ]:
df

,CountyNS,Coords,NDVI
0,00350547,"[-85.0425, 31.874166666666664]",0.0
1,00450376,"[-87.45777777777778, 41.01444444444445]",0.0
2,00465215,"[-93.79625, 40.69875]",0.0
3,01696228,"[-77.02933333333334, 39.553333333333335]",0.0
4,00758532,"[-89.7014285714286, 36.22380952380952]",0.0
...,...,...,...
3216,01155135,"[-117.21461538461536, 45.63025641025641]",0.0
3217,00345714,"[-82.442, 33.061]",0.0
3218,01419970,"[-135.4492156862745, 58.96215686274511]",0.0
3219,00069176,"[-91.57857142857144, 34.81238095238095]",0.0


## Query

In [ ]:
def get_query(t_start,t_end,coords):
  coord_str = [str(coords[1]),str(coords[0])]
  # poly_coord_str = "MultiPolygon (({}))".format(coords)
  query_json = {"layers": [
      # {
      #   "alias": "VIQA",
      #   "id": "57", # MODIS Aqua 16 days vegetation index quality asessment
      #   "output": "True",
      #   "temporal": {"intervals": [
      #       {"start": t_start.strftime(iso8601), "end": t_end.strftime(iso8601)}
      #   ]}, "type": "raster"
      # },
      {
        "alias": "NVDI",
        "id": "51", # MODIS Aqua 16 day normalized vegetation index
        "output": "True",
        "aggregation": "Max",
        "temporal": {"intervals": [
            {"start": t_start.strftime(iso8601), "end": t_end.strftime(iso8601)}
        ]}, 
       "type": "raster"
      }],
      "name": "result",
      "spatial" : {
          "type" : "point",
          "coordinates" : coord_str
          # "type" : "poly", 
          # "polygon" : {"wkt" : poly_coord_str}
      },
      "temporal": {"intervals": [
              {"start": t_start.strftime(iso8601), "end": t_end.strftime(iso8601)}
      ]}
  }
  return query_json

# timestamp

In [ ]:
# find the max after 1 year to ensure full converage of USA
year = 2021
month = 1
day = 1
hour = 0
timestamp = "{}-{}-{} {}:00".format(year,month,day,hour)
t_start = pd.Timestamp(timestamp)

year = 2022
month = 1
day = 1
hour = 0
timestamp = "{}-{}-{} {}:00".format(year,month,day,hour)
t_end = pd.Timestamp(timestamp)

iso8601 = '%Y-%m-%dT%H:%M:%SZ'

## Thread Query

In [ ]:
result_threaded = df.copy()

import traceback

def get_ndvi_forCounty(t_start,t_end,index,countyNS,coords):
    print('Getting NDVI for county number ', countyNS)
    try:
      query_json = get_query(t_start,t_end,coords)
      start_time = time.time()
      # # raster
      # query_result = query.submit_check_status_and_download(query_json)
      # point
      query_result = query.submit(query_json)
      point_df = query_result.point_data_as_dataframe()

      print('finished county {}: time {}'.format(countyNS,time.time() - start_time))
      # files = query_result.list_files()
    except:
      print('x'*50)
      print('ERROR WITH COUNTY ',countyNS)
      print(traceback.format_exc())
      print('X'*50)

    # # VIQA ~~~~~~~~~~~
    # VIQA_file =  [x for x in files if x.split(".")[-1] == 'csv' if 'VIQA' in x]
    # viqa = pd.read_csv(VIQA_file[0])
    # result.loc[index,'VIQA'] = viqa['mean()'].item()

    # # NDVI raster ~~~~~~~~~~~
    # NDVI_file =  [x for x in files if x.split(".")[-1] == 'tiff' if 'NDVI' in x]
    # array = np.array(PIL.Image.open(files[0]))
    # array[array==-9999]=np.nan
    # result_threaded.loc[index,'NDVI'] = np.nanmean(array)
    # time.sleep(2)
    # dir_2_rm = ("/").join(files[0].split("/")[0:4])
    # zip_2_rm = ("/").join(files[0].split("/")[0:4])+".zip"
    # shutil.rmtree(dir_2_rm)
    # os.remove(zip_2_rm)

    # NDVI point ~~~~~~~~~~~
    result_threaded.loc[index,'NDVI'] = round(float(point_df['value'][0]),3)



In [ ]:
files[0] = '/content/download/1675400400_40734969/16 day 250 m res imagery (NASA MODIS Aqua)-Normalized difference vegetation index (NDVI)[NVDI]-12_27_2021T00_00_00.tiff'
dir_2_rm = ("/").join(files[0].split("/")[0:4])
zip_2_rm = ("/").join(files[0].split("/")[0:4])+".zip"
shutil.rmtree(dir_2_rm)
os.remove(zip_2_rm)


In [ ]:
row = df.loc[0]
ountyNS = str(row['CountyNS'])
coords = row['Coords']
query_json = get_query(t_start,t_end,coords)
print(query_json)


# query_result = query.submit_check_status_and_download(query_json)

# files = query_result.list_files()
# NDVI_file =  [x for x in files if x.split(".")[-1] == 'tiff' if 'NDVI' in x]
# array = np.array(PIL.Image.open(files[0]))
# plt.figure(figsize = (20, 12))
# plt.imshow(array, cmap = 'Greens', vmin = -1, vmax = 1)
# plt.title('NDVI')
# plt.colorbar()
# plt.show()

<class 'list'>
{'layers': [{'alias': 'NVDI', 'id': '51', 'output': 'True', 'temporal': {'intervals': [{'start': '2021-01-01T00:00:00Z', 'end': '2022-01-01T00:00:00Z'}]}, 'type': 'raster'}], 'name': 'result', 'spatial': {'type': 'point', 'coordinates': '[-85.043,31.874]'}, 'temporal': {'intervals': [{'start': '2021-01-01T00:00:00Z', 'end': '2022-01-01T00:00:00Z'}]}}


In [ ]:
print(np.nanmean(array))

In [ ]:
files = query_result.list_files()
files
NDVI_file =  [x for x in files if x.split(".")[-1] == 'csv' if 'NDVI' in x]
ndvi = pd.read_csv(NDVI_file[0])
# result_threaded.loc[index,'NDVI'] = ndvi['mean()'].item()

In [ ]:
# clear all previously downloaded files
shutil.rmtree('/content/download')

In [ ]:

threadHandles = []

start_time = time.time()
for index, row in df.iterrows():
    if index < 500:
      continue
    countyNS = str(row['CountyNS'])
    coords = row['Coords']
    thread = threading.Thread(target=get_ndvi_forCounty,args=(t_start,t_end,index,countyNS,coords))
    threadHandles.append(thread)
    thread.start()
    time.sleep(2)

for thread in threadHandles:
  thread.join()

with open("/content/drive/My Drive/Colab Notebooks/CCUS_map/result_threaded_points.pkl", "wb") as f: # "wb" because we want to write in binary mode
    pickle.dump(result_threaded, f)
result_threaded.to_csv("/content/drive/My Drive/Colab Notebooks/CCUS_map/result_threaded_point.csv",index=False)

Getting NDVI for county number  00395066
2023-02-21 19:52:51 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 19:52:51 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 00395066: time 0.3248615264892578
Getting NDVI for county number  01383904
2023-02-21 19:52:53 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 19:52:53 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01383904: time 0.4498600959777832
Getting NDVI for county number  00161544
2023-02-21 19:52:55 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 19:52:55 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 00161544: time 0.5557107925415039
Getting NDVI for county number  00424239
2023-

Exception in thread Thread-822:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'value'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-82-5e5124b13088>", line 41, in get_ndvi_forCounty
  Fi

finished county 01480147: time 0.3484058380126953
Getting NDVI for county number  00758490
2023-02-21 20:02:20 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 20:02:20 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 00758490: time 0.4093756675720215
Getting NDVI for county number  00343365
2023-02-21 20:02:22 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 20:02:22 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 00343365: time 0.43233633041381836
Getting NDVI for county number  01804491
2023-02-21 20:02:24 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 20:02:24 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01804491: time 0.695847749710

Exception in thread Thread-1136:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'value'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-82-5e5124b13088>", line 41, in get_ndvi_forCounty
  F

finished county 01383821: time 0.37417149543762207
Getting NDVI for county number  00465262
2023-02-21 20:12:49 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 20:12:49 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 00465262: time 0.336012601852417
Getting NDVI for county number  01639801
2023-02-21 20:12:51 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 20:12:51 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01639801: time 0.6351103782653809
Getting NDVI for county number  01155133
2023-02-21 20:12:53 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 20:12:53 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01155133: time 0.3565669059753

...

finished county 00606927: time 0.3226504325866699
Getting NDVI for county number  00558495
2023-02-21 21:15:28 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 21:15:28 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 00558495: time 0.3858065605163574
Getting NDVI for county number  01384032
2023-02-21 21:15:30 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 21:15:30 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01384032: time 1.6528079509735107
Getting NDVI for county number  01448026
2023-02-21 21:15:32 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 21:15:32 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01448026: time 0.3281280994415

Exception in thread Thread-3165:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'value'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-82-5e5124b13088>", line 41, in get_ndvi_forCounty
  F

finished county 01804529: time 0.30870580673217773
Getting NDVI for county number  00450356
2023-02-21 21:20:35 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 21:20:35 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 00450356: time 0.32849931716918945
Getting NDVI for county number  01448035
2023-02-21 21:20:37 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 21:20:37 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01448035: time 0.3155336380004883
Getting NDVI for county number  01581075
2023-02-21 21:20:39 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 21:20:39 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01581075: time 0.90373849868

Exception in thread Thread-3233:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'value'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-82-5e5124b13088>", line 41, in get_ndvi_forCounty
  F

finished county 01622984: time 1.4430818557739258
Getting NDVI for county number  01383980
2023-02-21 21:22:51 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 21:22:51 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01383980: time 0.44423699378967285
Getting NDVI for county number  01639751
2023-02-21 21:22:53 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 21:22:53 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 01639751: time 0.36301350593566895
Getting NDVI for county number  00835840
2023-02-21 21:22:55 - paw - INFO - The '<class 'str'>' value 'True' was converted to a boolean 'True'.
2023-02-21 21:22:55 - paw - INFO - No running async even loop detected; performing asyncio.run() to start a new event loop.
finished county 00835840: time 0.38904690742

In [ ]:
print(result_threaded)

      CountyNS                                    Coords   NDVI
0     00350547            [-85.0425, 31.874166666666664]  0.883
1     00450376   [-87.45777777777778, 41.01444444444445]  0.854
2     00465215                     [-93.79625, 40.69875]  0.872
3     01696228  [-77.02933333333334, 39.553333333333335]  0.876
4     00758532    [-89.7014285714286, 36.22380952380952]  0.786
...        ...                                       ...    ...
3216  01155135  [-117.21461538461536, 45.63025641025641]  0.574
3217  00345714                         [-82.442, 33.061]  0.829
3218  01419970   [-135.4492156862745, 58.96215686274511]  0.477
3219  00069176   [-91.57857142857144, 34.81238095238095]  0.796
3220  01639733                        [-83.7455, 36.486]  0.917

[3221 rows x 3 columns]


In [ ]:
import pickle

with open("/content/drive/My Drive/Colab Notebooks/CCUS_map/result_threaded_500_1000.pkl", "wb") as f: # "wb" because we want to write in binary mode
    pickle.dump(result_threaded, f)

In [ ]:
print(len(result_threaded))

file_size = os.path.getsize('/content/download/1675400400_40734969.zip')
print(file_size/(1024*1024)*3221)

3221
101.17845344543457


In [ ]:
print(result_threaded[0:11])

    CountyNS                                             Coords      NDVI
0   00350547  (-85.14 31.84,-85.11 31.89,-85.07 31.97,-85.06...  0.614917
1   00450376  (-87.53 41.17,-87.39 41.16,-87.28 41.22,-87.27...  0.083851
2   00465215  (-94.01 40.90,-93.56 40.90,-93.56 40.58,-93.60...  0.048207
3   01696228  (-77.22 39.72,-77.00 39.72,-76.99 39.72,-76.79...  0.421020
4   00758532  (-89.96 36.39,-89.75 36.39,-89.73 36.43,-89.61...  0.338535
5   00974101  (-73.95 40.86,-73.94 40.87,-73.93 40.88,-73.92...  0.000000
6   00974144  (-74.26 42.80,-74.08 42.90,-74.09 42.96,-73.96...  0.000000
7   01247987  (-81.87 35.18,-81.77 35.18,-81.49 35.17,-81.37...  0.000000
8   01384033  (-103.33 32.00,-103.06 32.00,-103.06 32.09,-10...  0.000000
9   01008588  (-83.92 35.47,-83.85 35.52,-83.77 35.56,-83.66...  0.000000
10  00069902  (-92.42 35.79,-92.41 35.98,-92.31 35.97,-92.31...  0.000000
